## Health Insurance Issuers by County--Republican states offer fewer options

Using the Bloomberg data from this story: https://www.bloomberg.com/graphics/2017-health-insurer-exits/ matched with population data from Census and the Governor's party affiliation from the NGA.

In [1]:
import pandas as pd
import config   # File that contains api key
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

#### Collect data

In [ ]:
df = pd.read_csv('issuers_bloomberg_data.csv').query('year == 2017').set_index('year')
df = df.groupby('fips_county').issuer_name.count()

df2 = pd.read_csv('census_pop.csv').set_index('state').join(
    pd.read_csv('nga_gov_list.csv').set_index('state')).dropna().reset_index().set_index('fips_county')
data = df2.join(df).dropna()

In [ ]:
data.to_csv('clean_data.csv')

#### Basic calculations

In [ ]:
data['issuer_number'] = data['issuer_name']
d = {1: '1', 2: '2', 3: '3+', 4: '3+', 5: '3+', 6: '3+', 7: '3+', 8: '3+', 9: '3+', 10: '3+', 11: '3+', 12: '3+', 13: '3+'}
data = data.replace({'issuer_number': d})

In [ ]:
pd.pivot_table(data, values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

In [ ]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

In [ ]:
pd.pivot_table(data, values='pop', index=['issuer_number'], aggfunc=sum)

In [ ]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum).groupby(level=0).transform(lambda x: x/x.sum())

In [ ]:
len(data['state'].unique())

In [ ]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)#.groupby(level=0).transform(lambda x: x/x.sum())

In [ ]:
ll = pd.read_table('2015_Gaz_counties_national.txt').rename(columns={'GEOID': 'fips_county'}).set_index('fips_county')
ll['INTPTLONG'] = ll.iloc[:,-1].rename('INTPTLONG')
ll = ll[['INTPTLAT', 'INTPTLONG']]

In [ ]:
data = data.join(ll)

In [ ]:
data.to_csv('clean_data.csv')

In [ ]:
data

In [26]:
df = pd.read_csv('clean_data.csv').set_index('fips_county')

In [30]:
df = df[df['issuer_number'] == '1']

In [31]:
df['Governor'] = ['{} ({})'.format(row['gov_name'], row['gov_party'][0]) for index, row in df.iterrows()]

In [32]:
df['text'] = ['{}<br>Population: {:,}<br>Governor: {}'.format(
    row['county_name'], row['pop'], row['Governor']) for index, row in df.iterrows()]

In [44]:
df['size'] = df['pop']/50000 + 5

In [42]:
df

,state,county_name,pop,gov_name,gov_party,issuer_name,issuer_number,INTPTLAT,INTPTLONG,Governor,text,size,color
fips_county,,,,,,,,,,,,,
2013,AK,"Aleutians East Borough, Alaska",3296,Bill Walker,Independent,1.0,1,55.245044,-161.997477,Bill Walker (I),"Aleutians East Borough, Alaska<br>Population: ...",1.03296,#dacb4e
2016,AK,"Aleutians West Census Area, Alaska",5647,Bill Walker,Independent,1.0,1,51.959447,178.338813,Bill Walker (I),"Aleutians West Census Area, Alaska<br>Populati...",1.05647,#dacb4e
2020,AK,"Anchorage Municipality, Alaska",298192,Bill Walker,Independent,1.0,1,61.174250,-149.284329,Bill Walker (I),"Anchorage Municipality, Alaska<br>Population: ...",3.98192,#dacb4e
2050,AK,"Bethel Census Area, Alaska",17968,Bill Walker,Independent,1.0,1,60.929141,-160.152625,Bill Walker (I),"Bethel Census Area, Alaska<br>Population: 17,9...",1.17968,#dacb4e
2060,AK,"Bristol Bay Borough, Alaska",898,Bill Walker,Independent,1.0,1,58.730158,-156.996632,Bill Walker (I),"Bristol Bay Borough, Alaska<br>Population: 898...",1.00898,#dacb4e
2068,AK,"Denali Borough, Alaska",1953,Bill Walker,Independent,1.0,1,63.681106,-150.026544,Bill Walker (I),"Denali Borough, Alaska<br>Population: 1,953<br...",1.01953,#dacb4e
2070,AK,"Dillingham Census Area, Alaska",4954,Bill Walker,Independent,1.0,1,60.297320,-158.094726,Bill Walker (I),"Dillingham Census Area, Alaska<br>Population: ...",1.04954,#dacb4e
2090,AK,"Fairbanks North Star Borough, Alaska",100605,Bill Walker,Independent,1.0,1,64.692317,-146.601733,Bill Walker (I),"Fairbanks North Star Borough, Alaska<br>Popula...",2.00605,#dacb4e
2100,AK,"Haines Borough, Alaska",2496,Bill Walker,Independent,1.0,1,59.098771,-135.576936,Bill Walker (I),"Haines Borough, Alaska<br>Population: 2,496<br...",1.02496,#dacb4e


In [46]:
for index, row in df.iterrows():
    if row['gov_party'] == 'Republican':
        df.loc[index,'color'] = '#FF0000'
    if row['gov_party'] == 'Democratic':
        df.loc[index,'color'] = '#033192'
    if row['gov_party'] == 'Independent':
        df.loc[index,'color'] = '#f7dc6f'

In [63]:
trace1 = {
  "geo": "geo", 
  "hoverinfo": "text", 
  "lat": df['INTPTLAT'], 
  "lon": df['INTPTLONG'], 
  "marker": {
    "color": df['color'], 
    "line": {
      "color": "#202020", 
      "width": 0.8
    }, 
    "opacity": 0.5, 
    "size": df['size'], 
    "symbol": "circle"
  }, 
  "mode": "markers", 
  "name": "Counties with one health care exchange issuer", 
  "opacity": 1, 
  "showlegend": True, 
  "text": df['text'], 
  "type": "scattergeo", 
  "visible": True
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": 0.13, 
      "y": 1.05, 
      "align": "left", 
      "font": {"size": 20}, 
      "showarrow": False, 
      "text": "<b>Counties with only one health insurance provider on the state exchange</b>", 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.1, 
      "y": -0.1, 
      "align": "left", 
      "showarrow": False, 
      "text": "Hover to see details. Color is state governor political party: blue is Democratic, red is Republican, yellow is Independent. Circle size is population.  <br><b>Source:</b> Healthcare.gov and state exchanges", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": True, 
  "dragmode": "zoom", 
  "geo": {
    "landcolor": "rgb(234, 236, 238)",
    "lakecolor": "rgb( 138, 228, 255)",
    "bgcolor": "#fff", 
    "countrycolor": "#808080", 
    "countrywidth": 0.7, 
    "domain": {
      "x": [0, 1.1], 
      "y": [0, 1.1]
    }, 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 80], 
      "showgrid": False, 
      "tick0": 20
    }, 
    "lonaxis": {
      "dtick": 30, 
      "range": [-180, -50], 
      "showgrid": False, 
      "tick0": -180
    }, 
    "projection": {
      "scale": 1.03, 
      "type": "albers usa"
    }, 
    "resolution": 50, 
    "scope": "usa", 
    "showcountries": False, 
    "showlakes": True, 
    "showland": True, 
    "showrivers": False, 
    "showsubunits": True, 
    "subunitcolor": "#808080", 
    "subunitwidth": 0.7
  }, 
  "hidesources": False, 
  "hovermode": "closest", 
  "paper_bgcolor": "#fff", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "margin": {
    "r": 0, 
    "t": 20, 
    "b": 50, 
    "l": 0
  },
}
fig = Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='HC_County')
py.iplot(fig, filename='HC_County')

In [ ]:
tot = 51630721.0
d_sh = 10714794.0/tot
r_sh = 40182082.0/tot
tot_nc = 43030218.0
d_sh_nc = 10714794.0/tot_nc
r_sh_nc = 40182082.0/tot_nc

In [ ]:
d_sh_nc